In [13]:
library('igraph')

In [3]:
movement_graph <- read.table('../uber_data/movement_edgelist.txt')

In [4]:
g <- graph_from_data_frame(movement_graph, directed=FALSE)

In [6]:
E(g)$weight=as.numeric(movement_graph[,3])

In [7]:
is.connected(g)
writeLines(paste("Number of edges: ", ecount(g)))
writeLines(paste("Number of vertices: ", vcount(g)))
writeLines(paste("Diameter: ", diameter(g)))
#plot(g, vertex.label=NA, vertex.size=3)

[1] FALSE

Number of edges:  311812
Number of vertices:  1887
Diameter:  11911.84


In [11]:
dg <- decompose.graph(g)      
gcc_index <- which.max(sapply(dg,vcount))
gcc <- dg[[gcc_index]] 
clusters <- components(gcc)
print(dim(groups(clusters)))
writeLines(paste("Number of edges: ", ecount(gcc)))
writeLines(paste("Number of vertices: ", vcount(gcc)))
writeLines(paste("Diameter of GCC: ", diameter(gcc)))

[1] 1
Number of edges:  311802
Number of vertices:  1880
Diameter of GCC:  11911.84
